## Task 4

In [1]:
# Read the data and get a quick overview
data3 <- read.csv("data_ca3.csv")
head(data3)

,x,y,z,v,n
,<int>,<int>,<int>,<int>,<int>
1,0,1,0,1,40
2,0,0,0,1,315
3,0,1,0,0,9
4,0,0,0,0,50
5,0,1,1,0,6
6,0,0,1,0,24


Next, we need to prepare the data before fitting a logistic model. That is, we convert each variable to dummy variables.

In [2]:
# Convert the variables into dummy variables
data3$x <- as.factor(data3$x)
data3$y <- as.factor(data3$y)
data3$z <- as.factor(data3$z)
data3$v <- as.factor(data3$v)

The next step is to select a good logistic regression model. To do this, we start by defining the full model and then implement a stepwise approach to select the best model relative to AIC.

In [3]:
# Define the full model including all interactions
model_full <- glm(
    v ~ x * y * z, family = binomial(link = "logit"), data = data3, weights = n
)

# Model selection using stepwise in both directions
model_step <- step(model_full, direction = "both")

Start:  AIC=1099.27
v ~ x * y * z

        Df Deviance    AIC
- x:y:z  1   1083.6 1097.6
<none>       1083.3 1099.3

Step:  AIC=1097.63
v ~ x + y + z + x:y + x:z + y:z

        Df Deviance    AIC
- y:z    1   1083.9 1095.9
- x:y    1   1084.0 1096.0
- x:z    1   1084.1 1096.1
<none>       1083.6 1097.6
+ x:y:z  1   1083.3 1099.3

Step:  AIC=1095.86
v ~ x + y + z + x:y + x:z

       Df Deviance    AIC
- x:y   1   1084.2 1094.2
- x:z   1   1084.4 1094.4
<none>      1083.9 1095.9
+ y:z   1   1083.6 1097.6

Step:  AIC=1094.22
v ~ x + y + z + x:z

       Df Deviance    AIC
- x:z   1   1084.8 1092.8
<none>      1084.2 1094.2
- y     1   1086.7 1094.7
+ x:y   1   1083.9 1095.9
+ y:z   1   1084.0 1096.0

Step:  AIC=1092.76
v ~ x + y + z

       Df Deviance    AIC
<none>      1084.8 1092.8
- y     1   1087.2 1093.2
+ x:z   1   1084.2 1094.2
+ x:y   1   1084.4 1094.4
+ y:z   1   1084.5 1094.5
- x     1   1091.3 1097.3
- z     1   1422.4 1428.4


The selected model only lowers the AIC by 7.1, which is marginally better than the full model. In contrast, simpler models are generally preferred. However, the main focus is to analyze how smoking habits influence the child survival. Thus, we include the two-way interaction terms `x:y` and `y:z`.

$$
\text{logit}\bigl(P(\text{Child survives}) \bigr) = \beta_0 + \beta_1X + \beta_2Y +\beta_3Z + \beta_4XY + \beta_5YZ.
$$

For easier interpretation, we generate a table with relevant results.

In [5]:
# Define the model
model_simplified <- glm(
    v ~ x + y + z + x:y + y:z, family = binomial(link = "logit"), data = data3, weights = n
                       )
# Extract coefficients, standard errors, and p-values

summary_model <- summary(model_simplified)

coef_table <- summary_model$coefficients
coef_df <- data.frame(
  Coefficient = round(coef_table[, "Estimate"], digits = 4),
  Std_Error = round(coef_table[, "Std. Error"], digits = 4),
  P_value = round(coef_table[, "Pr(>|z|)"], digits = 4)
)

# Calculate odds ratios and 95% confidence intervals
coef_df$Odds_Ratio <- round(exp(coef_df$Coefficient), digits = 4)
coef_df$CI_Lower <- round(exp(coef_df$Coefficient - 1.96 * coef_df$Std_Error), digits = 4)
coef_df$CI_Upper <- round(exp(coef_df$Coefficient + 1.96 * coef_df$Std_Error), digits = 4)

# Rearrange columns for better readability
coef_df <- coef_df[, c("Coefficient", "Std_Error", "Odds_Ratio", "CI_Lower", "CI_Upper", "P_value")]

coef_df 

,Coefficient,Std_Error,Odds_Ratio,CI_Lower,CI_Upper,P_value
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.8158,0.1401,6.1460,4.6702,8.0881,0.0000
x1,-0.5050,0.1912,0.6035,0.4149,0.8779,0.0083
y1,-0.4122,0.3686,0.6622,0.3215,1.3638,0.2634
z1,3.3494,0.1995,28.4856,19.2667,42.1157,0.0000
x1:y1,0.3537,0.5911,1.4243,0.4472,4.5370,0.5496
y1:z1,-0.2967,0.5301,0.7433,0.2630,2.1008,0.5757


From the table above we can deduce the following

1. **Main Effects:**

   - **Mother's Age $(x_1)$:** Significant on the level $1\%$, with an estimated odds ratio of $0.6035$, indicating that children born to mothers aged 30 or older have about $40\%$ lower odds of survival compared to mothers under 30, controlling for other factors.

   - **Smoking Habits $(y_1)$:** Not significant, with an estimated odds ratio of $0.6622$. This indicates that smoking habits alone do not significantly affect child survival.

   - **Gestational Age $(z_1)$:** Highly significant ($p < 0.0001$), with a large odds ratio of $28.4856$, showing that longer gestational age is strongly associated with higher odds of survival.

2. **Interaction Effects:**

   - **Mother's Age and Smoking Habits $(x_1:y_1)$:** Not significant, suggesting that the effect of smoking habits on child survival does not depend on gestational age.

   - **Smoking Habits and Gestational Age $(y_1:z_1)$:** Not significant, suggesting that the effect of smoking habits on child survival does not depend on gestational age.
  
### Smoking Habits and Child Survival

The coefficient for $y_1$ $(\beta_2 = -0.4122)$ suggests that smoking 5+ cigarettes per day is associated with reduced odds of child survival (odds ratio = 0.6622), but this reduction is not statistically significant. Furthermore, the $95\%$ C.I for the odds ratio $[0.3215, 1.3638]$ includes 1, indicating that the effect could range from a positive effect to a detrimental one. This reinforces the lack of strong evidence for a consistent relationship between smoking and child survival in this dataset. 

The results are rather surprising, since smoking is usually associatied with health risks.

#### Possible Limitations

- Sparse counts? There are clear imbalance in the number of counts for smoking categories. Even though we weight with the number of counts, the imbalance could still limit the power to detect an effect. A possible solution to the imbalance could be to aggregate the data for smoking habits, such as combining the smoking categories into a single binary variable (e.g. Y = 1 smokes, Y = 0, does not smoke). This could help with a more even spread in counts, depending on the population the data was sampled from.

- Multicolinearity? There might be some multicolinearity between the predictors, which could explain the reason as to why smoking habits are non-significant.